# SVD Plots

### Importing Libraries

In [46]:
from keras import backend as K
from keras import regularizers
import numpy as np
from gensim_download import pickle_rw
from keras.layers import Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
import plotly.plotly as py
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy.linalg



from numpy.linalg import det

from numpy.linalg import inv

import sympy as sympy

from sympy import Matrix
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot



### Defining Functions

In [15]:
def make_dict(vocab, vectors):
    """Make dictionary of vocab and vectors"""
    return {vocab[i]: vectors[i] for i in range(len(vocab))}


def vocab_train_test(embedding, lg1, lg2, lg1_vocab):
    """Create training and test vocabularies"""
    if embedding == 'zeroshot':
        with open('../data/zeroshot/transmat/data/' +
                  'OPUS_en_it_europarl_train_5K.txt') as f:
            vocab_train = [(_.split(' ')[0], _.split(' ')[1])
                           for _ in f.read().split('\n')[:-1]]
        with open('../data/zeroshot/transmat/data/' +
                  'OPUS_en_it_europarl_test.txt') as f:
            vocab_test = [(_.split(' ')[0], _.split(' ')[1])
                          for _ in f.read().split('\n')[:-1]]

    elif embedding in ['fasttext_random', 'fasttext_top']:
        embedding, split = embedding.split('_')
        lg1_lg2, lg2_lg1 = pickle_rw((lg1 + '_' + lg2, 0),
                                     (lg2 + '_' + lg1, 0), write=False)
        # T = Translation, R = Reverse (translated and then translated back)
        # Create vocab from 2D translations
        vocab_2D = []
        for lg1_word in lg1_vocab:
            # Translate lg1_word
            if lg1_word in lg1_lg2:
                lg1_word_T = lg1_lg2[lg1_word]

                # Check if translated word (or lowercase) is in lg2_lg1
                if lg1_word_T in lg2_lg1.keys():
                    lg1_word_R = lg2_lg1[lg1_word_T]
                elif lg1_word_T.lower() in lg2_lg1.keys():
                    lg1_word_T = lg1_word_T.lower()
                    lg1_word_R = lg2_lg1[lg1_word_T]
                else:
                    lg1_word_R = None

                # Check if lg1_word and lg1_word_R are equal (lowercase)
                if lg1_word_R:
                    if lg1_word.lower() == lg1_word_R.lower():
                        vocab_2D.append((lg1_word, lg1_word_T))
        print('length of '+ lg1+'-'+ lg2+ ' vocab: '+str(len(vocab_2D)))

        #Create Train/Test vocab

        if split == 'random':
            sample = np.random.choice(len(vocab_2D), 6500, replace=False)
            vocab_train = np.asarray(vocab_2D)[sample[:5000]].tolist()
            vocab_test = np.asarray(vocab_2D)[sample[5000:]].tolist()
        elif split == 'top':
            sample = np.random.choice(range(6500), 6500, replace=False)
            vocab_train = np.asarray(vocab_2D)[:5000, :].tolist()
            vocab_test = np.asarray(vocab_2D)[:1500, :].tolist()
        else:
            pass

        # if split == 'random':
        #     sample = np.random.choice(len(vocab_2D), 900, replace=False)
        #     vocab_train = np.asarray(vocab_2D)[sample[:700]].tolist()
        #     vocab_test = np.asarray(vocab_2D)[sample[700:]].tolist()
        # elif split == 'top':
        #     sample = np.random.choice(range(900), 900, replace=False)
        #     vocab_train = np.asarray(vocab_2D)[:700, :].tolist()
        #     vocab_test = np.asarray(vocab_2D)[:200, :].tolist()
        # else:
        #     pass

    return vocab_train, vocab_test


def vectors_train_test(vocab_train, vocab_test):
    """Create training and test vectors"""
    X_train, y_train = zip(*[(lg1_dict[lg1_word], lg2_dict[lg2_word])
                             for lg1_word, lg2_word in vocab_train])
    X_test, y_test = zip(*[(lg1_dict[lg1_word], lg2_dict[lg2_word])
                           for lg1_word, lg2_word in vocab_test])
    return map(np.asarray, (X_train, X_test, y_train, y_test))


def translation_matrix(X_train, y_train):
    """Fit translation matrix T"""
    model = Sequential()
    model.add(Dense(300, use_bias=False, input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(0.01)))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(X_train, y_train, batch_size=128, epochs=20,
                        verbose=False)
    T = model.get_weights()[0]

    T = np.matrix(T)

    M = np.multiply(np.matrix(T),100)

    T_norm, T_normed = normalize(M)

    D = np.linalg.det(M)
    
    I = inv(T)
    
    #Fr_norm = np.linalg.det(np.subtract(np.matmul(M,M.getH()),np.matmul(M.getH(),M)))
    
    Fr_norm = np.linalg.det(np.matrix(np.subtract(np.matmul(T,T.getH()),np.matmul(T.getH(),T))))
    
    #print(T_normed)
    #print(T_normed.getH())

    #print(np.around(np.matmul(T_normed,T_normed.getH())))

    #print(np.around(np.matmul(T_normed.getH(),T_normed)))

    print ("Determinant:"+str(D))
    
    #print ("Fr_norm:"+str(Fr_norm))

    if np.array_equal(np.around(np.matmul(T_normed,T_normed.getH())), np.around(np.matmul(T_normed.getH(),T_normed))) == True:
        tf = "True"
    else:
        tf = "False"

    return model, history, T, D, tf, I, M

def translation_accuracy(X_test, y_test):
    """Get predicted matrix 'yhat' using 'T' and find translation accuracy"""
    # yhat
    yhat = X.dot(T)
    count = 0
    for i in range(len(y_test)):
        if yhat[i,:].all() == y_test[i,:].all():
            count = count + 1
    accuracy = count/len(y_test)*100
    return accuracy

def svd(T):
    """Perform SVD on the translation matrix 'T' """
    U, s, Vh = numpy.linalg.svd(T, full_matrices=False )
    return U, s, Vh

def T_svd_EDA(s):
    """Perform SVD on the translation matrix 'T' """
    plt.hist(s, bins='auto', range = (0,1),normed = 1)
    plt.show()




def normalize(matrix):
    """Normalize the rows of a matrix"""
    matrix_norm = np.linalg.norm(matrix, axis=1)
    matrix_normed = matrix / np.repeat(matrix_norm, matrix.shape[1]). \
        reshape(matrix.shape)
    return matrix_norm, matrix_normed


def translation_results(X, y, vocab, M, lg2_vectors, lg2_vocab):
    """X, y, vocab - The training or test data that you want results for
    T - The translation matrix
    lg2_vectors, lg2_vocab - Foreign language used to find the nearest neighbor
    """

    # Data Prep on Inputs
    X_word, y_word = zip(*vocab)
    X_norm, X_normed = normalize(X)
    y_norm, y_normed = normalize(y)
    lg2_vectors_norm, lg2_vectors_normed = normalize(lg2_vectors)

    # yhat
    yhat = X.dot(M)
    yhat_norm, yhat_normed = normalize(yhat)

    #X_norm = normalize(X)


    # Nearest Neighbors
#     neg_cosine = -yhat_normed.dot(lg2_vectors_normed.T)
#     ranked_neighbor_indices = np.argsort(neg_cosine, axis=1)

#     # Nearest Neighbor
#     nearest_neighbor_indices = ranked_neighbor_indices[:, 0]
#     yhat_neighbor = lg2_vectors[nearest_neighbor_indices, :]
#     yhat_neighbor_norm, yhat_neighbor_normed = normalize(yhat_neighbor)
#     yhat_neighbor_word = np.asarray(lg2_vocab)[nearest_neighbor_indices]

#     # Results DF
#     cols = ['X_norm', 'y_norm', 'yhat_norm', 'yhat_neighbor_norm',
#             'X_word', 'y_word', 'yhat_neighbor_word']
#     results_df = pd.DataFrame({'X_norm': X_norm,
#                                'y_norm': y_norm,
#                                'yhat_norm': yhat_norm,
#                                'yhat_neighbor_norm': yhat_neighbor_norm,
#                                'X_word': X_word,
#                                'y_word': y_word,
#                                'yhat_neighbor_word': yhat_neighbor_word,})
#     results_df = results_df[cols]
#     results_df['neighbor_correct'] = results_df.y_word == \
#         results_df.yhat_neighbor_word

    return yhat_norm


def T_norm_EDA(results_df):
    """Plot result norms side-by-side"""
    test_size = results_df.shape[0]
    test_accuracy = round(results_df.neighbor_correct.mean(), 2)

    print('Test Accuracy: '+str(test_accuracy)+'\n')

    plot_data = ['X_norm', 'y_norm', 'yhat_norm', 'yhat_neighbor_norm']
    # f, ax = plt.subplots(len(plot_data), sharex=True, sharey=True,
    #                      figsize=(10, 10))
    # for i, d in enumerate(plot_data):
    #     ax[i].hist(results_df[d], bins=100)
    #     ax[i].axis('off')
    #     title = '{}: mean={}, std={}'.format(d, round(results_df[d].mean(), 2), round(results_df[d].std(), 2))
    #     ax[i].set_title(title)
    # f.subplots_adjust(hspace=0.7)
    # plt.savefig('../images/' + lg1 + '_' + lg2 + '_' + embedding +
    #             '_T_norm.png')
    # plt.close('all')
    return


def T_pca_EDA(T):
    """PCA on matrix T"""
    T_ss = StandardScaler().fit_transform(T)
    pca = PCA().fit(T_ss)
    n = pca.n_components_

    # plt.figure(figsize=(10, 6))
    # plt.xlim((0, n))
    # plt.ylim((0, 1))
    # plt.plot(range(n + 1), [0] + np.cumsum(pca.explained_variance_ratio_).
    #          tolist())
    # plt.plot(range(n + 1), np.asarray(range(n + 1)) / n)
    # plt.xlabel('Number of Eigenvectors')
    # plt.ylabel('Explained Variance')
    # plt.savefig('../images/' + lg1 + '_' + lg2 + '_' + embedding +
    #             '_T_isotropy.png')
    # plt.close('all')

    isotropy = (1 - sum(np.cumsum(pca.explained_variance_ratio_) * 1 / n)) / .5
    return isotropy


def T_report_results(embedding, lg1, lg2, lg1_vectors, lg2_vectors,
                     X_train, X_test, D, results_df, isotropy):
    md = '## ' + lg1.title() + ' to ' + lg2.title() + ' ' + \
        embedding.title() + '  \n'
    md += '- ' + lg1.title() + ' Vocabulary Size = ' + \
        '{:,.0f}'.format(lg1_vectors.shape[0]) + '  \n'
    md += '- ' + lg1.title() + ' Embedding Length = ' + \
        '{:,.0f}'.format(lg1_vectors.shape[1]) + '  \n'
    md += '- ' + lg2.title() + ' Vocabulary Size = ' + \
        '{:,.0f}'.format(lg2_vectors.shape[0]) + '  \n'
    md += '- ' + lg2.title() + ' Embedding Length = ' + \
        '{:,.0f}'.format(lg2_vectors.shape[1]) + '  \n'
    md += '- Train Size = ' + '{:,.0f}'.format(X_train.shape[0]) + '  \n'
    md += '- Test Size = ' + '{:,.0f}'.format(X_test.shape[0]) + '  \n'
    md += '- Determinant = ' + '{:,.0f}'.format(D) + '  \n'

    md += '- <b>Test Accuracy = ' + \
        '{:,.1%}'.format(results_df.neighbor_correct.mean()) + '</b>  \n\n'



    md += '#### Test L2 Norms  \n'
    md += '- X_norm: L2 norms for ' + lg1.title() + ' test vectors  \n'
    md += '- y_norm: L2 norms for ' + lg2.title() + ' test vectors  \n'
    md += '- yhat_norm: L2 norms for X.dot(T) test vectors ' + \
        '(T = translation matrix)  \n'
    md += '- yhat_neighbor norm: L2 norms for nearest neighbor' + \
        'to X.dot(T) in y test vectors  \n'
    md += '![](../images/' + lg1 + '_' + lg2 + '_' + embedding + \
        '_T_norm.png)  \n\n'

    md += '#### Translation Matrix Isotropy  \n'
    md += '- Isotropy = ' + '{:,.1%}'.format(isotropy) + '  \n'
    md += '![](../images/' + lg1 + '_' + lg2 + '_' + embedding + \
        '_T_isotropy.png)  \n\n'

    return md

### Main Function ( with SVD stats )

In [36]:
if __name__ == '__main__':
    # Manually set list of translations (embedding, lg1, lg2)
    translations = [#('fasttext_random', 'en', 'ru'),
                    ('fasttext_top', 'en', 'en'),
                    ('fasttext_top', 'en', 'ru'),
                    #('fasttext_random', 'en', 'de'),
                    ('fasttext_top', 'en', 'de'),
                    #('fasttext_random', 'en', 'es'),
                    ('fasttext_top', 'en', 'es'),
                    #('fasttext_random', 'en', 'zh-CN'),
                    ('fasttext_top', 'en', 'zh-CN'),
        
        
        
                     #('fasttext_random', 'ru', 'en'),
                    ('fasttext_top', 'ru', 'en'),
                    #('fasttext_random', 'ru', 'es'),
                    ('fasttext_top', 'ru', 'ru'),
                    ('fasttext_top', 'ru', 'de'),
                    ('fasttext_top', 'ru', 'es'),
                    #('fasttext_random', 'ru', 'zh-CN'),
                    ('fasttext_top', 'ru', 'zh-CN'),
                    #('fasttext_random', 'ru', 'de'),
                    
        
        
                    #('fasttext_random', 'de', 'en'),
                    ('fasttext_top', 'de', 'en'),
                    #('fasttext_random', 'de', 'es'),
                    ('fasttext_top', 'de', 'ru'), 
                    #('fasttext_random', 'de', 'ru'),
                    ('fasttext_top', 'de', 'de'),
                    ('fasttext_top', 'de', 'es'),
                    #('fasttext_random', 'de', 'zh-CN'),
                    ('fasttext_top', 'de', 'zh-CN'),
        
        
        
                    #('fasttext_random', 'es', 'en'),
                    ('fasttext_top', 'es', 'en'),
                    #('fasttext_random', 'es', 'de'),
                    ('fasttext_top', 'es', 'ru'),
                    ('fasttext_top', 'es', 'de'),
                    #('fasttext_random', 'es', 'ru'),
                    ('fasttext_top', 'es', 'es'),
                    #('fasttext_random', 'es', 'zh-CN'),
                    ('fasttext_top', 'es', 'zh-CN'),
        
                
        
                    #('fasttext_random', 'zh-CN', 'en'),
                    ('fasttext_top', 'zh-CN', 'en'),
                    #('fasttext_random', 'zh-CN', 'es'),
                    ('fasttext_top', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'de'),
                    ('fasttext_top', 'zh-CN', 'es'),
                    #('fasttext_random', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'zh-CN'),
                    #('fasttext_random', 'zh-CN', 'de'),
        
                    
                    

                    ]
   
    s_min_en=[]
    s_max_en=[]
    s_mean_en=[]
    s_median_en=[]
    s_std_en=[]
            
    s1_min_en=[]
    s1_max_en=[]
    s1_mean_en=[]
    s1_median_en=[]
    s1_std_en=[]
        
       
        
    s_min_es=[]
    s_max_es=[]
    s_mean_es=[]
    s_median_es=[]
    s_std_es=[]
            
    s1_min_es=[]
    s1_max_es=[]
    s1_mean_es=[]
    s1_median_es=[]
    s1_std_es=[]
            
       
    s_min_ru=[]
    s_max_ru=[]
    s_mean_ru=[]
    s_median_ru=[]
    s_std_ru=[]
            
    s1_min_ru=[]
    s1_max_ru=[]
    s1_mean_ru=[]
    s1_median_ru=[]
    s1_std_ru=[]
            
       
        
    s_min_de=[]
    s_max_de=[]
    s_mean_de=[]
    s_median_de=[]
    s_std_de=[]
        
    s1_min_de=[]
    s1_max_de=[]
    s1_mean_de=[]
    s1_median_de=[]
    s1_std_de=[]
        
      
        
    s_min_zh=[]
    s_max_zh=[]
    s_mean_zh=[]
    s_median_zh=[]
    s_std_zh=[]
            
    s1_min_zh=[]
    s1_max_zh=[]
    s1_mean_zh=[]
    s1_median_zh=[]
    s1_std_zh=[]

    md = ''
    for translation in translations:
        embedding, lg1, lg2 = translation
        # Vocab/Vectors/Dicts
        lg1_vocab, lg1_vectors, lg2_vocab, lg2_vectors = \
            pickle_rw((lg1 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg1 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      write=False)
        lg1_dict = make_dict(lg1_vocab, lg1_vectors)
        lg2_dict = make_dict(lg2_vocab, lg2_vectors)

        print('Translation: '+lg1+'->'+lg2+'\n')

        # Train/Test Vocab/Vectors
        vocab_train, vocab_test = vocab_train_test(embedding, lg1, lg2, lg1_vocab)
        X_train, X_test, y_train, y_test = vectors_train_test(vocab_train,
                                                              vocab_test)

        # Fit tranlation matrix to training data
        model, history, T, D, tf,I, M = translation_matrix(X_train, y_train)

        print('biggest element:'+str(np.max(T))+'\n')
        
        U,s,Vh = svd(T)
        
        #scaler = MinMaxScaler()
        
        #scaler.fit(s)
        
        #s = scaler.transform(s)
        
        s1 = np.log10(s)
        
        print("min: "+str(min(s))+"\n")
        
        print("max: "+str(max(s))+"\n")
        
        print("mean: "+str(np.mean(s))+"\n")
        
        print("median: "+str(np.median(s))+"\n")
        
        print("std: "+str(np.std(s))+"\n")
        
        
        
        print("min: "+str(min(s1))+"\n")
        
        print("max: "+str(max(s1))+"\n")
        
        print("mean: "+str(np.mean(s1))+"\n")
        
        print("median: "+str(np.median(s1))+"\n")
        
        print("std: "+str(np.std(s1))+"\n")
        
        
        if lg1 == 'en':
            s_min_en.append(min(s))
            s_max_en.append(max(s))
            s_mean_en.append(np.mean(s))
            s_median_en.append(np.median(s))
            s_std_en.append(np.std(s))
            
            s1_min_en.append(min(s1))
            s1_max_en.append(max(s1))
            s1_mean_en.append(np.mean(s1))
            s1_median_en.append(np.median(s1))
            s1_std_en.append(np.std(s1))
        
        if lg1 == 'es':
        
            s_min_es.append(min(s))
            s_max_es.append(max(s))
            s_mean_es.append(np.mean(s))
            s_median_es.append(np.median(s))
            s_std_es.append(np.std(s))
            
            s1_min_es.append(min(s1))
            s1_max_es.append(max(s1))
            s1_mean_es.append(np.mean(s1))
            s1_median_es.append(np.median(s1))
            s1_std_es.append(np.std(s1))
            
        if lg1 == 'ru':
        
            s_min_ru.append(min(s))
            s_max_ru.append(max(s))
            s_mean_ru.append(np.mean(s))
            s_median_ru.append(np.median(s))
            s_std_ru.append(np.std(s))
            
            s1_min_ru.append(min(s1))
            s1_max_ru.append(max(s1))
            s1_mean_ru.append(np.mean(s1))
            s1_median_ru.append(np.median(s1))
            s1_std_ru.append(np.std(s1))
            
        if lg1 == 'de':
        
            s_min_de.append(min(s))
            s_max_de.append(max(s))
            s_mean_de.append(np.mean(s))
            s_median_de.append(np.median(s))
            s_std_de.append(np.std(s))
            
            s1_min_de.append(min(s1))
            s1_max_de.append(max(s1))
            s1_mean_de.append(np.mean(s1))
            s1_median_de.append(np.median(s1))
            s1_std_de.append(np.std(s1))
       
        if lg1 == 'zh-CN':
        
            s_min_zh.append(min(s))
            s_max_zh.append(max(s))
            s_mean_zh.append(np.mean(s))
            s_median_zh.append(np.median(s))
            s_std_zh.append(np.std(s))
            
            s1_min_zh.append(min(s1))
            s1_max_zh.append(max(s1))
            s1_mean_zh.append(np.mean(s1))
            s1_median_zh.append(np.median(s1))
            s1_std_zh.append(np.std(s1))
    
      
        
        
        
        
#         #print(s)
        
#         plt.hist(s,bins='auto')#bins=50,normed='True',range = (0.0,0.2))
#         #plt.plot(s)
#         plt.show()
        
        

Translation: en->en

length of en-en vocab: 20000
Determinant:inf
biggest element:0.0531415

min: 0.00067539

max: 0.522189

mean: 0.0215776

median: 0.0150824

std: 0.0359025

min: -3.17045

max: -0.282172

mean: -1.83782

median: -1.82153

std: 0.361513

Translation: en->ru

length of en-ru vocab: 6839
Determinant:-1.60411e-20
biggest element:0.0220535

min: 1.21853e-05

max: 0.66592

mean: 0.0188323

median: 0.00972134

std: 0.0436656

min: -4.91416

max: -0.176578

mean: -2.06598

median: -2.01227

std: 0.580684

Translation: en->de

length of en-de vocab: 16693
Determinant:2.33429e-17
biggest element:0.0246625

min: 0.000102474

max: 0.639408

mean: 0.018553

median: 0.00982725

std: 0.0421359

min: -3.98939

max: -0.194222

mean: -2.05544

median: -2.00757

std: 0.552418

Translation: en->es

length of en-es vocab: 10422
Determinant:-9.69121e-24
biggest element:0.022694

min: 9.48394e-06

max: 0.564666

mean: 0.0172508

median: 0.00970471

std: 0.0375242

min: -5.02301

max: -0.2

AttributeError: 'NoneType' object has no attribute 'append'

## Heatmaps of SVD Statistics

In [53]:
    s_min = []
    s_min.append(s_min_en)
    s_min.append(s_min_ru)
    s_min.append(s_min_de)
    s_min.append(s_min_es)
    s_min.append(s_min_zh)
    s_max = []
    s_max.append(s_max_en)
    s_max.append(s_max_ru)
    s_max.append(s_max_de)
    s_max.append(s_max_es)
    s_max.append(s_max_zh)
    s_mean = []
    s_mean.append(s_mean_en)
    s_mean.append(s_mean_ru)
    s_mean.append(s_mean_de)
    s_mean.append(s_mean_es)
    s_mean.append(s_mean_zh)
    s_median = []
    s_median.append(s_median_en)
    s_median.append(s_median_ru)
    s_median.append(s_median_de)
    s_median.append(s_median_es)
    s_median.append(s_median_zh)
    s_std = []
    s_std.append(s_std_en)
    s_std.append(s_std_ru)
    s_std.append(s_std_de)
    s_std.append(s_std_es)
    s_std.append(s_std_zh)
        
    s1_min = []
    s1_min.append(s1_min_en)
    s1_min.append(s1_min_ru)
    s1_min.append(s1_min_de)
    s1_min.append(s1_min_es)
    s1_min.append(s1_min_zh)
    s1_max = []
    s1_max.append(s1_max_en)
    s1_max.append(s1_max_ru)
    s1_max.append(s1_max_de)
    s1_max.append(s1_max_es)
    s1_max.append(s1_max_zh)
    s1_mean = []
    s1_mean.append(s1_mean_en)
    s1_mean.append(s1_mean_ru)
    s1_mean.append(s1_mean_de)
    s1_mean.append(s1_mean_es)
    s1_mean.append(s1_mean_zh)
    s1_median = []
    s1_median.append(s1_median_en)
    s1_median.append(s1_median_ru)
    s1_median.append(s1_median_de)
    s1_median.append(s1_median_es)
    s1_median.append(s1_median_zh)
    s1_std = []
    s1_std.append(s1_std_en)
    s1_std.append(s1_std_ru)
    s1_std.append(s1_std_de)
    s1_std.append(s1_std_es)
    s1_std.append(s1_std_zh)
    
    init_notebook_mode(connected=True)

    
    
    

    trace = go.Heatmap(z=s_min,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s_min',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_min')
    
    trace = go.Heatmap(z=s_max,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s_max',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_max')
    
    trace = go.Heatmap(z=s_mean,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s_mean',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_mean')
    
    trace = go.Heatmap(z=s_median,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s_median',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_median')
    
    trace = go.Heatmap(z=s_std,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    fig = go.Figure(data=data, layout=layout)
    layout = go.Layout(
    title='s_std',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_std')
    
    
    
    
    
    
    trace = go.Heatmap(z=s1_min,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s1_min',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s1_min')
    
    trace = go.Heatmap(z=s1_max,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s_max',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s_max')
    
    trace = go.Heatmap(z=s1_mean,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s1_mean',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s1_mean')
    
    trace = go.Heatmap(z=s1_median,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s1_median',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s1_median')
    
    trace = go.Heatmap(z=s1_std,
                   x=['en', 'ru', 'de', 'es', 'zh-CN'],
                   y=['en', 'ru', 'de', 'es', 'zh-CN'])
    data=[trace]
    layout = go.Layout(
    title='s1_std',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='s1_std')
    
                                     

## Inverted Matrices

In [11]:
if __name__ == '__main__':
    # Manually set list of translations (embedding, lg1, lg2)
    translations = [#('fasttext_random', 'en', 'ru'),
                    ('fasttext_top', 'en', 'ru'),
                    #('fasttext_random', 'en', 'de'),
                    ('fasttext_top', 'en', 'de'),
                    #('fasttext_random', 'en', 'es'),
                    ('fasttext_top', 'en', 'es'),
                    #('fasttext_random', 'en', 'zh-CN'),
                    ('fasttext_top', 'en', 'zh-CN'),
                    #('fasttext_random', 'de', 'en'),
                    ('fasttext_top', 'de', 'en'),
                    #('fasttext_random', 'de', 'es'),
                    ('fasttext_top', 'de', 'es'),
                    #('fasttext_random', 'de', 'ru'),
                    ('fasttext_top', 'de', 'ru'),
                    #('fasttext_random', 'de', 'zh-CN'),
                    ('fasttext_top', 'de', 'zh-CN'),
                    #('fasttext_random', 'ru', 'en'),
                    ('fasttext_top', 'ru', 'en'),
                    #('fasttext_random', 'ru', 'es'),
                    ('fasttext_top', 'ru', 'es'),
                    #('fasttext_random', 'ru', 'zh-CN'),
                    ('fasttext_top', 'ru', 'zh-CN'),
                    #('fasttext_random', 'ru', 'de'),
                    ('fasttext_top', 'ru', 'de'),
                    #('fasttext_random', 'zh-CN', 'en'),
                    ('fasttext_top', 'zh-CN', 'en'),
                    #('fasttext_random', 'zh-CN', 'es'),
                    ('fasttext_top', 'zh-CN', 'es'),
                    #('fasttext_random', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'ru'),
                    #('fasttext_random', 'zh-CN', 'de'),
                    ('fasttext_top', 'zh-CN', 'de'),
                    #('fasttext_random', 'es', 'en'),
                    ('fasttext_top', 'es', 'en'),
                    #('fasttext_random', 'es', 'de'),
                    ('fasttext_top', 'es', 'de'),
                    #('fasttext_random', 'es', 'ru'),
                    ('fasttext_top', 'es', 'ru'),
                    #('fasttext_random', 'es', 'zh-CN'),
                    ('fasttext_top', 'es', 'zh-CN')

                    ]

    md = ''
    for translation in translations:
        embedding, lg1, lg2 = translation
        # Vocab/Vectors/Dicts
        lg1_vocab, lg1_vectors, lg2_vocab, lg2_vectors = \
            pickle_rw((lg1 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg1 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      write=False)
        lg1_dict = make_dict(lg1_vocab, lg1_vectors)
        lg2_dict = make_dict(lg2_vocab, lg2_vectors)

        print('Translation: '+lg1+'->'+lg2+'\n')

        # Train/Test Vocab/Vectors
        vocab_train, vocab_test = vocab_train_test(embedding, lg1, lg2, lg1_vocab)
        X_train, X_test, y_train, y_test = vectors_train_test(vocab_train,
                                                              vocab_test)

        # Fit tranlation matrix to training data
        model, history, T, D, tf, I, M = translation_matrix(X_train, y_train)

        print('Inverse:'+str(I)+'\n')
        
        
        
        

Translation: en->ru

length of en-ru vocab: 6839
Determinant:6.68409e-20
Fr_norm:-0.0
Inverse:[[  11.04030704  -58.05263901  -70.07519531 ...,   25.41628647
    30.97108078  -40.67876816]
 [  91.81015015  155.81185913   63.31090164 ...,  -51.2548027   -34.41306305
    47.90806961]
 [  23.46608734   12.1979866   -52.79821777 ...,   23.93603897
    -7.66915035    5.51542711]
 ..., 
 [ -95.40709686  -29.07827187   87.29953003 ...,  100.88738251  -16.7738266
   153.37922668]
 [-147.87860107  -81.68314362  -44.3213768  ...,  173.213974    -59.09873962
    59.42921829]
 [  16.56924438 -103.65172577 -142.10444641 ...,  -78.63152313
   117.04570007 -168.28567505]]

Translation: en->de

length of en-de vocab: 16693
Determinant:2.40427e-21
Fr_norm:0.0
Inverse:[[  755.14819336  -133.340271     206.62522888 ...,  1882.01904297
   -709.5010376  -1083.3581543 ]
 [  242.50422668   -23.07047462     5.77775288 ...,   960.47021484
   -322.10214233  -420.7321167 ]
 [ -431.28991699    97.79253387   -95.18

Translation: zh-CN->es

length of zh-CN-es vocab: 1164
Determinant:-0.0
Fr_norm:0.0
Inverse:[[  17.54487991   23.9071846   -69.86971283 ...,   89.22011566
    67.72798157  108.96387482]
 [ -19.26591682  -58.13566971  -63.12947464 ...,   12.89937305  102.1413269
    64.83384705]
 [  53.17620087 -105.45384216  -46.82474899 ...,   29.39961052
   285.44174194   51.69458771]
 ..., 
 [ -97.70675659 -172.0501709    11.54387856 ...,  -69.68674469
   -87.34493256  -99.78017426]
 [ 132.74069214  -67.03653717 -117.35790253 ...,   83.51582336
   427.69055176  193.97187805]
 [ -33.97097397  -28.83506012   35.82284927 ...,   70.83348083
   -68.67223358   97.35033417]]

Translation: zh-CN->ru

length of zh-CN-ru vocab: 1266
Determinant:-0.0
Fr_norm:-0.0
Inverse:[[ -37.29304123   18.05200195  120.03998566 ..., -134.6721344    97.88696289
    -2.2412765 ]
 [  81.02592468   48.94137955 -144.85075378 ...,   -4.63170481
   -69.93479919   10.25763702]
 [  61.70194244 -108.61361694  -82.55732727 ...,  -18.1

## Sanity Check on Determinants

In [19]:
if __name__ == '__main__':
    # Manually set list of translations (embedding, lg1, lg2)
    translations = [#('fasttext_random', 'en', 'ru'),
                    ('fasttext_top', 'en', 'ru'),
                    #('fasttext_random', 'en', 'de'),
                    ('fasttext_top', 'en', 'de'),
                    #('fasttext_random', 'en', 'es'),
                    ('fasttext_top', 'en', 'es'),
                    #('fasttext_random', 'en', 'zh-CN'),
                    ('fasttext_top', 'en', 'zh-CN'),
                    #('fasttext_random', 'de', 'en'),
                    ('fasttext_top', 'de', 'en'),
                    #('fasttext_random', 'de', 'es'),
                    ('fasttext_top', 'de', 'es'),
                    #('fasttext_random', 'de', 'ru'),
                    ('fasttext_top', 'de', 'ru'),
                    #('fasttext_random', 'de', 'zh-CN'),
                    ('fasttext_top', 'de', 'zh-CN'),
                    #('fasttext_random', 'ru', 'en'),
                    ('fasttext_top', 'ru', 'en'),
                    #('fasttext_random', 'ru', 'es'),
                    ('fasttext_top', 'ru', 'es'),
                    #('fasttext_random', 'ru', 'zh-CN'),
                    ('fasttext_top', 'ru', 'zh-CN'),
                    #('fasttext_random', 'ru', 'de'),
                    ('fasttext_top', 'ru', 'de'),
                    #('fasttext_random', 'zh-CN', 'en'),
                    ('fasttext_top', 'zh-CN', 'en'),
                    #('fasttext_random', 'zh-CN', 'es'),
                    ('fasttext_top', 'zh-CN', 'es'),
                    #('fasttext_random', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'ru'),
                    #('fasttext_random', 'zh-CN', 'de'),
                    ('fasttext_top', 'zh-CN', 'de'),
                    #('fasttext_random', 'es', 'en'),
                    ('fasttext_top', 'es', 'en'),
                    #('fasttext_random', 'es', 'de'),
                    ('fasttext_top', 'es', 'de'),
                    #('fasttext_random', 'es', 'ru'),
                    ('fasttext_top', 'es', 'ru'),
                    #('fasttext_random', 'es', 'zh-CN'),
                    ('fasttext_top', 'es', 'zh-CN')

                    ]

    md = ''
    for translation in translations:
        embedding, lg1, lg2 = translation
        # Vocab/Vectors/Dicts
        lg1_vocab, lg1_vectors, lg2_vocab, lg2_vectors = \
            pickle_rw((lg1 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg1 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      write=False)
        lg1_dict = make_dict(lg1_vocab, lg1_vectors)
        lg2_dict = make_dict(lg2_vocab, lg2_vectors)

        print('Translation: '+lg1+'->'+lg2+'\n')

        # Train/Test Vocab/Vectors
        vocab_train, vocab_test = vocab_train_test(embedding, lg1, lg2, lg1_vocab)
        X_train, X_test, y_train, y_test = vectors_train_test(vocab_train,
                                                              vocab_test)
        
        

        # Fit tranlation matrix to training data
        model, history, T, D, tf, I, M = translation_matrix(X_train, y_train)

        
        y_norm = translation_results(X_test, y_test, vocab_test, T,
                                        lg2_vectors, lg2_vocab)
        
        
        #print('Volume of X matrix: '+ np.prod(X_norm)+'\n')
        print(y_norm)
        print(T)
        print('Volume of yhat matrix: '+ str(np.prod(y_norm))+'\n')
    
        

Translation: en->ru

length of en-ru vocab: 6839
Determinant:2.03775e-19
[ 0.67061174  0.6873163   0.85272115 ...,  1.20578921  1.40582168
  0.93844253]
[[  4.34536341e-04  -1.01042946e-03   1.29217806e-04 ...,   2.51814048e-03
    2.25269631e-03  -3.78022756e-04]
 [ -2.70378281e-04   1.11204310e-04   3.61257058e-04 ...,   1.47847389e-03
    2.56023044e-03   3.06686998e-04]
 [ -7.82030053e-04  -2.08674697e-03  -3.18972440e-03 ...,   1.43284816e-03
    8.56383296e-04  -5.04361931e-04]
 ..., 
 [  3.68732936e-03   1.84817414e-03   2.45620660e-03 ...,   4.04990744e-03
   -1.71988842e-03  -2.58150208e-03]
 [  9.60162492e-04   5.78932850e-05   2.89971940e-03 ...,   8.86483584e-04
   -2.09363061e-03  -8.01478338e-04]
 [ -3.22616356e-03  -2.79117911e-03  -1.80032675e-03 ...,   1.22218125e-03
   -1.76331308e-03  -7.65946519e-04]]
Volume of yhat matrix: inf

Translation: en->de

length of en-de vocab: 16693
Determinant:-1.29322e-21
[ 0.70435685  0.65216172  0.62163788 ...,  1.12678242  1.3214262

KeyboardInterrupt: 